# Pico W Microcontroller Basic Usage

Here, we assume that the Pico W is already set up and running a web server on the same
network as this computer (i.e. the one you're running this notebook on). We're going to
interact with our web server using the `requests` library.

We need to know what data to send via POST to our web server. We can find this out by
opening up dev tools (e.g. `F12` or `Ctrl+Shift+I` on Google Chrome) and navigating to the network tab. From
there, we can enter RGB values and click the "Send command to LED" button, at which
point the POST and GET requests will be shown as follows.

Open dev tools:
![Dev tools](pico-w-server-dev-tools.png)

Click button:
![Dev tools click](pico-w-server-dev-tools-click.png)

Payload:
![Dev tools click, payload](pico-w-server-dev-tools-click-post.png)

Now, we use the payload information and pass that in via a `post` method as a `data` kwarg.

In [1]:
import requests

# cookies = {'rgb': "{'blue': 22, 'red': 20, 'green': 21}"}
# r = requests.post('http://192.168.0.111/', cookies=cookies)
payload = {
    "control_led": "Send+command+to+LED",
    "blue": "22",
    "red": "20",
    "green": "21",
}
r = requests.post("http://192.168.0.111/", data=payload)
r

<Response [200]>

We can also access the sensor data via one of the cookies that gets set via the web server.

In [2]:
r.cookies["sensor_data"]

"{'ch670': 11, 'ch720': 2, 'ch480': 29, 'ch445': 17, 'ch615': 5, 'ch560': 4, 'ch415': 1, 'ch515': 10}"

To read this as a dictionary, we can use the (safe) `literal_eval` function.

In [3]:
import ast
ast.literal_eval(r.cookies["sensor_data"])

{'ch670': 11,
 'ch720': 2,
 'ch480': 29,
 'ch445': 17,
 'ch615': 5,
 'ch560': 4,
 'ch415': 1,
 'ch515': 10}

Now that we can send commands to the LED and read the sensor data, let's define some
helper functions. Also note that with the MicroPython NeoPixel interface, there is no overall
"brightness" parameter, just the individual brightnesses for red, green, and blue.

In [4]:
import requests
import ast
def observe_sensor_data(R, G, B, url="http://192.168.0.111/"):
    payload = {
        "control_led": "Send+command+to+LED",
        "red": str(R),
        "green": str(G),
        "blue": str(B),
    }
    r = requests.post(url, data=payload)
    sensor_data_cookie = r.cookies["sensor_data"]
    return ast.literal_eval(sensor_data_cookie)

In [5]:
sensor_data = observe_sensor_data(10, 10, 10)
sensor_data

{'ch670': 4,
 'ch720': 1,
 'ch480': 21,
 'ch445': 8,
 'ch615': 2,
 'ch560': 3,
 'ch415': 0,
 'ch515': 4}

In [6]:
observe_sensor_data(0, 0, 0)

{'ch670': 0,
 'ch720': 1,
 'ch480': 18,
 'ch445': 4,
 'ch615': 1,
 'ch560': 3,
 'ch415': 0,
 'ch515': 2}

The observation function has been exposed as a parameter for `SelfDrivingLabDemo`.

In [7]:
%load_ext autoreload
%autoreload 2 # just some IPython magic to recognize changes to installed packages
import pandas as pd
from self_driving_lab_demo.core import SelfDrivingLabDemo

In [8]:
sdl = SelfDrivingLabDemo(
    simulation=False,
    autoload=True,
    observe_sensor_data_fn=observe_sensor_data,
    observe_sensor_data_kwargs=dict(url="http://192.168.0.111/"),
)


In [9]:
sdl.evaluate(10, 20, 30)

{'ch670': 5,
 'ch720': 2,
 'ch480': 35,
 'ch445': 28,
 'ch615': 3,
 'ch560': 4,
 'ch415': 1,
 'ch515': 9,
 'mae': 7.375,
 'rmse': 13.024016277631105,
 'frechet': 33.0}

In [10]:
sim = SelfDrivingLabDemo(simulation=True, autoload=True)
sim.evaluate(10, 20, 30)

{'ch410': 0.015574136996242369,
 'ch440': 0.4130776771289992,
 'ch470': 1.853812225451396,
 'ch510': 0.4671727581176467,
 'ch550': 0.13300714858610418,
 'ch583': 0.018261464102564014,
 'ch620': 0.18066150431255917,
 'ch670': 0.0027292799607842966,
 'mae': 0.1357450410986569,
 'rmse': 0.2349491005585053,
 'frechet': 0.4523043231158368}